In [31]:
import numpy as np 
import os
#import skimage.io as io
#import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

#custom
import tensorflow as tf

#from new tutorial
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

import random
import cv2
import tifffile as tiff

In [152]:
datapath = "/mnt/hgfs/VMsharedFolder/git/misc"
x_path = "/processed/val/images/"
y_path = "/processed/val/masks/"
#x_train1 = cv2.imread(datapath+'/RGB-PanSharpen_AOI_2_Vegas_img991.tif' ,1)
x_train0 = tiff.imread(datapath+'/RGB-PanSharpen_AOI_2_Vegas_img991.tif')
x_train = x_train0[:512 ,:512 ,:]
x_train = (x_train > 0).astype(np.int32)
x_train1 = tf.convert_to_tensor(x_train ,np.float32)
x_train2 = tf.expand_dims(x_train1, axis=-1)
y_train0 = np.load(datapath+y_path+'/991_mask.npy')
y_train1 = y_train0[:512 ,:512]
y_train = (y_train1 > 0).astype(np.int32)

print("x_train:")
print(x_train.dtype)
print(x_train.shape)
print(type(x_train))
print("\nx_train1:")
print(x_train1.dtype)
print(x_train1.shape)
print(type(x_train1))
print("\nx_train2:")
print(x_train2.dtype)
print(x_train2.shape)
print(type(x_train2))
print("\ny_train:")
print(y_train.dtype)
print(y_train.shape)
print(type(y_train))
print("\ny_train1:")
print(y_train1.dtype)
print(y_train1.shape)
print(type(y_train1))

img_size = x_train.shape
print(img_size)

x_train:
int32
(512, 512, 3)
<class 'numpy.ndarray'>

x_train1:
<dtype: 'float32'>
(512, 512, 3)
<class 'tensorflow.python.framework.ops.EagerTensor'>

x_train2:
<dtype: 'float32'>
(512, 512, 3, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>

y_train:
int32
(512, 512)
<class 'numpy.ndarray'>

y_train1:
uint8
(512, 512)
<class 'numpy.ndarray'>
(512, 512, 3)


In [4]:
x_train10 = x_train[:256 ,:256 ,:]
y_train10 = y_train[:256 ,:256]
img_size10 = x_train10.shape
num_classes = 2
print(x_train10.shape)
print(y_train10.shape)

(256, 256, 3)
(256, 256)


In [ ]:
epochs = 15

In [156]:
num_classes = 2
batch_size = 2

input_img_paths = datapath + x_path
target_dir = datapath + y_path

x=[]
y=[]

input_img_paths = sorted(
    [
        os.path.join(input_img_paths ,fname)
        for fname in os.listdir(input_img_paths)
       # if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
       # if fname.endswith(".png") and not fname.startswith(".")
        
    ]
)

for npy in input_img_paths:
    x.append(np.load(npy))

for npy in target_img_paths:
    y.append(np.load(npy))
    
x = np.array(x)
x = x [: ,:512 ,:512 ,:]
y = np.array(y)
y = y[: ,:512 ,:512]
print(x.shape)
print(y.shape)

(5, 512, 512, 3)
(5, 512, 512)


num_classes = 2
batch_size = 2

input_img_paths = datapath + x_path
target_dir = datapath + y_path

input_img_paths = sorted(
    [
        os.path.join(input_img_paths ,fname)
        for fname in os.listdir(input_img_paths)
       # if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
       # if fname.endswith(".png") and not fname.startswith(".")
        
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)
    x_train = tiff.imread(datapath+'/RGB-PanSharpen_AOI_2_Vegas_img991.tif')

from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

# Display input image #7
display(Image(filename=input_img_paths[2]))

# Display auto-contrast version of corresponding target (per-pixel categories)
img = PIL.ImageOps.autocontrast(load_img(target_img_paths[2]))
display(img)

from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
            y[j] -= 1
        return x, y

In [153]:
from tensorflow.keras import layers

def get_model(img_size, num_classes):
    #inputs = keras.Input(shape=img_size + (3,))
    inputs = keras.Input(shape=img_size)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
#model = get_model((650,650,3), num_classes)
model = get_model(img_size, 2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Split our img paths into a training and a validation set
val_samples = 2
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = OxfordPets(
    batch_size, img_size, train_input_img_paths, train_target_img_paths
)
val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [157]:
train_size = 1
x_train = x[train_size:]
x_val = x[:train_size]
y_train = y[train_size:] 
y_val = y[:train_size]
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(4, 512, 512, 3)
(1, 512, 512, 3)
(4, 512, 512)
(1, 512, 512)


In [158]:
# initialize the number of epochs and batch size
EPOCHS = 100
BS = 1
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

H = model.fit(x_train, y_train
            #  ,batch_size=BS
              ,validation_data=(x_val, y_val)
              ,steps_per_epoch=len(x_train) // BS
              ,epochs=EPOCHS)

Epoch 1/100
4/4 [==============================] - 8s 1s/step - loss: 5.0672 - val_loss: 438.7365
Epoch 2/100
4/4 [==============================] - 4s 969ms/step - loss: 1.3749 - val_loss: 37.1716
Epoch 3/100
4/4 [==============================] - 4s 952ms/step - loss: 0.4942 - val_loss: 9.1968
Epoch 4/100
4/4 [==============================] - 4s 908ms/step - loss: 0.8299 - val_loss: 15.6864
Epoch 5/100
4/4 [==============================] - 4s 912ms/step - loss: 0.2390 - val_loss: 33.8303
Epoch 6/100
4/4 [==============================] - 4s 921ms/step - loss: 0.3930 - val_loss: 12.6906
Epoch 7/100
4/4 [==============================] - 4s 909ms/step - loss: 0.3955 - val_loss: 8.1710
Epoch 8/100
4/4 [==============================] - 4s 939ms/step - loss: 0.1994 - val_loss: 3.7833
Epoch 9/100
4/4 [==============================] - 4s 929ms/step - loss: 0.2310 - val_loss: 5.4902
Epoch 10/100
4/4 [==============================] - 4s 942ms/step - loss: 0.1441 - val_loss: 7.1169
Epoch 

Epoch 83/100
4/4 [==============================] - 4s 933ms/step - loss: 0.0155 - val_loss: 0.3576
Epoch 84/100
4/4 [==============================] - 4s 919ms/step - loss: 0.0205 - val_loss: 0.2862
Epoch 85/100
4/4 [==============================] - 4s 926ms/step - loss: 0.0214 - val_loss: 0.3357
Epoch 86/100
4/4 [==============================] - 4s 922ms/step - loss: 0.0361 - val_loss: 0.2546
Epoch 87/100
4/4 [==============================] - 4s 985ms/step - loss: 0.0321 - val_loss: 0.3172
Epoch 88/100
4/4 [==============================] - 4s 1s/step - loss: 0.0393 - val_loss: 0.3403
Epoch 89/100
4/4 [==============================] - 4s 991ms/step - loss: 0.0216 - val_loss: 0.2700
Epoch 90/100
4/4 [==============================] - 4s 936ms/step - loss: 0.0167 - val_loss: 0.2851
Epoch 91/100
4/4 [==============================] - 4s 936ms/step - loss: 0.0455 - val_loss: 0.3257
Epoch 92/100
4/4 [==============================] - 4s 966ms/step - loss: 0.0180 - val_loss: 0.2955
Epo